In [1]:
import requests
from lxml import html
import json

In [2]:
url = "https://www.immobilienscout24.de/Suche/S-2/Wohnung-Miete/Baden-Wuerttemberg/Karlsruhe/-/-/25,00-/EURO--800,00"

In [177]:
def fetch_apartment(url):
    response = requests.get(url)
    tree = html.fromstring(response.content)
    xpath_from_date = '//dd[@class="is24qa-bezugsfrei-ab grid-item three-fifths"]/text()'
    xpath_charges = '//dd[@class="is24qa-nebenkosten grid-item three-fifths"]/text()'
    xpath_price_total = '//dd[@class="is24qa-gesamtmiete grid-item three-fifths font-bold"]/text()'
    return {
        "from_date": tree.xpath(xpath_from_date)[0].strip() if len(tree.xpath(xpath_from_date)) else 'N/A', 
        "charges": tree.xpath(xpath_charges)[1].strip() if len(tree.xpath(xpath_charges)) else 'N/A',
        "price_total": tree.xpath(xpath_price_total)[0].strip() if len(tree.xpath(xpath_price_total)) else 'N/A',
    }
        

def fetch_list(url):
    response = requests.get(url)
    tree = html.fromstring(response.content)
    latest = tree.xpath('//div[@class="result-list-entry__data"]')
    result = []
    ids = []
    for apartment in latest:
        id = apartment.xpath('./a[@class="result-list-entry__brand-title-container"]')[0].get('data-go-to-expose-id')
        price, area, rooms = [_.xpath('./dd//text()')[0] for _ in apartment.xpath('.//dl')]
        url_apartment = f'https://www.immobilienscout24.de/expose/{id}'
        
        dict_ = {
            "id": id,
            "title": apartment.xpath('./a[@class="result-list-entry__brand-title-container"]/h5/text()')[0],
            "url": url_apartment,
            "address": apartment.xpath('.//button/div/text()')[0],
            "price": price, 
            "area": area,
            "rooms": rooms,
        }
    
        dict_.update(fetch_apartment(url_apartment))
            
        result.append(dict_)
        ids.append(id)
    
    return {
        "results": result,
        "ids": ids,
    }

### Output

In [178]:
fetch_list(url)

{'results': [{'id': '70500125',
   'title': 'KA - Südstadt: Schöne 2 Zimmerwohnung',
   'url': 'https://www.immobilienscout24.de/expose/70500125',
   'address': 'Südstadt, Karlsruhe',
   'price': '505 €',
   'area': '47 m²',
   'rooms': '2',
   'from_date': '15.12.2019',
   'charges': '80 €',
   'price_total': '585 € (zzgl. Heizkosten)'},
  {'id': '114370494',
   'title': 'Ideal für Wochenendheimfahrer/-in! Gemütliche 1-Zimmerwohnung am Rande von Karlsruhe-Rüppurr!',
   'url': 'https://www.immobilienscout24.de/expose/114370494',
   'address': 'Rüppurr, Karlsruhe',
   'price': '210 €',
   'area': '28 m²',
   'rooms': '1',
   'from_date': 'Kurzfristig frei!',
   'charges': '90 €',
   'price_total': '300 €'},
  {'id': '113862142',
   'title': 'City-Park Karlsruhe helle schöne, renovierte 2-Zimmer-Wohnung mit SW Balkon + EBK + TG-Platz',
   'url': 'https://www.immobilienscout24.de/expose/113862142',
   'address': 'Elisabeth-Großwendt-Straße 2, Südstadt, Karlsruhe',
   'price': '720 €',
   

In [179]:
res = fetch_list(url)

In [182]:
res['ids']

['70500125',
 '114370494',
 '113862142',
 '114368747',
 '114364608',
 '72953749',
 '114358718',
 '114346741',
 '57819755',
 '114338963',
 '114333709',
 '114330666',
 '114320083',
 '114319613',
 '72945471',
 '114317958',
 '114313588',
 '114309910',
 '114288805',
 '114302177']